In [ ]:
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes

In [2]:
import random 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline 
from sklearn.cluster import KMeans 
import requests 
from pandas.io.json import json_normalize 
import folium 
from bs4 import BeautifulSoup
import lxml

In [3]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find('table')
df = pd.read_html(str(table))[0]

df.replace({'Not assigned':np.NaN}, inplace = True)
df.replace({'/':','}, inplace = True)
df.dropna(subset=['Borough'], inplace = True)
df.reset_index(drop = True, inplace = True)
df.head(12)



,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
df.shape

(103, 3)

In [15]:
import geocoder
lat_lng_coords = None
lat_lon = pd.DataFrame({'lat':[],'lon':[]})


In [31]:
table = df.set_index('Postal code').join(geo_coord.set_index('Postal Code'))
table.head()

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [33]:
map_Can = folium.Map(location=[43.7181557,-79.5181421], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(table['Latitude'], table['Longitude'], table['Borough'], table['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Can)  
    
map_Can

Used analysis similar to NY example. Seems there is a Gaussian distribution originating in downtown Toronto.